<a href="https://colab.research.google.com/github/Piyush150398/Fine-Tuning-LLM/blob/main/FineTuning_using_SFT_for_Financial_Sentiment_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lets Import the Data set from DeepLake

In [ ]:
pip install deeplake trl peft transformers accelerate wandb

In [ ]:
import deeplake

ds = deeplake.load('hub://genai360/FingGPT-sentiment-train-set')

ds_valid = deeplake.load('hub://genai360/FingGPT-sentiment-valid-set') # validation set

print(ds)

/

Opening dataset in read-only mode as you don't have write permissions.


-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/genai360/FingGPT-sentiment-train-set



-

...

Opening dataset in read-only mode as you don't have write permissions.


\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/genai360/FingGPT-sentiment-valid-set



\

hub://genai360/FingGPT-sentiment-valid-set loaded successfully.

Dataset(path='hub://genai360/FingGPT-sentiment-train-set', read_only=True, tensors=['input', 'instruction', 'output'])


In [ ]:
ds.summary()

Dataset(path='hub://genai360/FingGPT-sentiment-train-set', read_only=True, tensors=['input', 'instruction', 'output'])

   tensor      htype     shape      dtype  compression
   -------    -------   -------    -------  ------- 
    input      text    (20000, 1)    str     None   
 instruction   text    (20000, 1)    str     None   
   output      text    (20000, 1)    str     None   


In [ ]:
ds.visualize()

HINT: Please forward the port - 40855 to your local machine, if you are running on the cloud.
 * Serving Flask app 'dataset_visualizer'
 * Debug mode: off


# **Lets Look into data-set**

In [ ]:
# Input tweet
print(ds.input[7].text())

# Input Instruction
print(ds.instruction[7].text())

# Output
print(ds.output[7].text())

In December , Amer announced the dismissal of CEO Roger Talermo , who had headed the company since 2006 .
What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
neutral


In [ ]:
def prepare_sample_text(example):
    """Prepare the text from a sample of the dataset."""
    text = f"{example['instruction'].text()}\n\nContent: {example['input'].text()}\n\nSentiment: {example['output'].text()}"
    return text

print(prepare_sample_text(ds[:,0]))

What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}

Content: Tesla Motors recalls 2,700 Model X SUVs https://t.co/BFWS3DbM0U $TSLA

Sentiment: negative


In [ ]:
# Load the tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

# create the constant length dataset
from trl.trainer import ConstantLengthDataset

# Need to import the constant length dataset for training
train_dataset = ConstantLengthDataset(
    tokenizer,   # select tokenizer
    ds,          # select dataset
    formatting_func=prepare_sample_text, # select formatting function
    infinite=True,
    seq_length=1024 # which means the input sequences fed to the model during training will have a length of 1024 tokens
)

eval_dataset = ConstantLengthDataset(
    tokenizer,
    ds_valid,
    formatting_func=prepare_sample_text,
    seq_length=1024
)

# Show one sample from train set
iterator = iter(train_dataset)
sample = next(iterator)
print(sample)
train_dataset.start_iteration = 0

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:548: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictionary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


{'input_ids': tensor([  35, 1313,    2,  ...,    2,    2, 2264]), 'labels': tensor([  35, 1313,    2,  ...,    2,    2, 2264])}


In [ ]:
# If having following error then run this cell
# ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`
import os
os._exit(00)

In [ ]:
# Define LoRA configuration
from peft import LoraConfig

loar_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Define Traning Arguments
# The TrainingArguments class is part of the Hugging Face Transformers library and is commonly used to configure training settings for machine learning models.
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./OPT-fine_tuned-FinGPT-CPU",  # Directory where model checkpoints and predictions will be saved
    dataloader_drop_last=True,  # Drops the last incomplete batch if the dataset size is not divisible by the batch size
    evaluation_strategy="epoch",  # Evaluate every epoch
    save_strategy="epoch",  # Save model every epoch
    num_train_epochs=10,  # Total number of training epochs
    logging_steps=5,  # Log training information every 5 steps
    per_device_train_batch_size=12,  # Batch size per GPU/TPU core/CPU for training
    per_device_eval_batch_size=12,  # Batch size per GPU/TPU core/CPU for evaluation
    learning_rate=1e-4,  # Learning rate for the optimizer
    lr_scheduler_type="cosine",  # Cosine learning rate scheduler
    warmup_steps=100,  # Number of warmup steps for the learning rate scheduler
    gradient_accumulation_steps=1,  # Number of steps for gradient accumulation before performing a backward/update pass
    gradient_checkpointing=False,  # Use gradient checkpointing to reduce memory usage during training
    fp16=False,  # Use 16-bit (half-precision) floating-point precision for training
    bf16=True,  # Use 16-bit (bfloat16) floating-point precision for training
    weight_decay=0.05,  # Strength of weight decay regularization
    ddp_find_unused_parameters=False,  # Distributed Data Parallel (DDP) option to find and report unused parameters
    run_name="OPT-fine_tuned-FinGPT-CPU",  # Name of the training run (for logging purposes)
    report_to="wandb",  # Report training progress to Weights & Biases (wandb)
)



In [ ]:
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-1.3b", torch_dtype=torch.bfloat16
)

### This code is preparing a pre-trained language model for further training by freezing most of its parameters, ensuring stability for certain parameters, enabling gradient checkpointing, and customizing the output layer for potential downstream tasks. The focus seems to be on training adapters or other task-specific components while keeping the majority of the model fixed.

In [ ]:
from torch import nn

# Loop through all parameters in the model
for param in model.parameters():
    param.requires_grad = False  # Freeze the model parameters - train adapters later (This is done to prevent these parameters from being updated during training, as indicated by the comment.)
    if param.ndim == 1:
        # Cast small parameters (e.g., layernorm) to fp32 for stability
        param.data = param.data.to(torch.float32)

# Enable gradient checkpointing to reduce the number of stored activations
# a technique used to reduce the memory requirements during backpropagation by recomputing intermediate activations as needed.
model.gradient_checkpointing_enable()

# Enable input gradients to be required (useful for training adapters later)
"""This line enables input gradients to be required, which is useful for training adapters later. Adapters are additional small neural networks that can be trained on specific downstream tasks while keeping the rest of the pre-trained model frozen."""
model.enable_input_require_grads()

# Define a custom module to cast the output to fp32
class CastOutputToFloat(nn.Sequential):
    def forward(self, x):
        return super().forward(x).to(torch.float32)

# Replace the lm_head (language modeling head) of the model with the custom CastOutputToFloat module
model.lm_head = CastOutputToFloat(model.lm_head)


### Now, connect the model, dataset, training arguments, and Lora config together using the SFTTrainer class to start the training process by invoking the .train() method.

In [ ]:
from trl import SFTTrainer

# Initialize the SFTTrainer with the required parameters
trainer = SFTTrainer(
    model=model,               # The pre-trained language model to be fine-tuned
    args=training_args,        # Training arguments for the trainer
    train_dataset=train_dataset,  # Training dataset
    eval_dataset=eval_dataset,    # Evaluation dataset
    peft_config=loar_config,      # Self-paced fine-tuning (SFT) configuration, possibly for learning rates or other hyperparameters
    packing=True,                # Enable packing, a technique for efficient training using smaller batches
)

print("Training...")
trainer.train()


Training...


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


# **Merging LoRA and OPT**

### Before conducting inference and observing the results, the final task is to load the LoRA adaptors from the preceding stage and merge them with the base model.

In [ ]:
# Load the base model (OPT-1.3B) using AutoModelForCausalLM from the Transformers library
from transformers import AutoModelForCausalLM
import torch

# Load the pre-trained OPT-1.3B model for causal language modeling
model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-1.3b",   # Model name or path to the pre-trained model
    return_dict=True,       # Return outputs as a dictionary
    torch_dtype=torch.bfloat16  # Specify the data type to be used (bfloat16 in this case)
)

# Load the LoRA adaptors
from peft import PeftModel

# Load the LoRA model from a specific checkpoint
model = PeftModel.from_pretrained(
    model,   # The base model (OPT-1.3B) loaded earlier
    "./OPT-fine_tuned-FinGPT-CPU/<desired_checkpoint>/",   # Path to the directory containing the LoRA checkpoint. This file will be in the formate 'adapter_config.json'.
)

# Set the model to evaluation mode
model.eval()

# Merge the LoRA adaptors with the base model and unload the adaptors
model = model.merge_and_unload()

# Save the merged model for future use
model.save_pretrained("./OPT-fine_tuned-FinGPT-CPU/merged")

# **Inference**

### We randomly selected four previously unseen examples from the dataset and provided them as input to both the vanilla base model (OPT-1.3B) and the fine-tuned model in order to contrast their respective responses. The code is relatively straightforward when utilizing the .generate() method from the Transformers library.

In [ ]:
# Tokenize the input text using the tokenizer
inputs = tokenizer(
    """What is the sentiment of this news? Please choose an answer from
    {strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive},
    then provide some short reasons.\n\n
    Content: UPDATE 1-AstraZeneca sells rare cancer drug to Sanofi for up to S300 mln.\n\nSentiment: """,
    return_tensors="pt"  # Return PyTorch tensors
).to("cuda:0")  # Move tensors to GPU (cuda:0)

# Generate text using the pre-trained language model
generation_output = model.generate(
    **inputs,  # Pass the tokenized inputs
    return_dict_in_generate=True,  # Return outputs as a dictionary
    output_scores=True,  # Return the scores of the generated tokens
    max_length=256,  # Maximum length of the generated sequence
    num_beams=1,  # Number of beams for beam search
    do_sample=True,  # Use sampling for generation
    repetition_penalty=1.5,  # Repetition penalty to discourage repeated tokens
    length_penalty=2.  # Length penalty to encourage longer sequences
)

# Decode the generated sequence and print the result
print(tokenizer.decode(generation_output['sequences'][0]))
